In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import DataframeBuilder

In [5]:
target_type_list = ["TotalPrice", "TotalPartsSold"]
presence_type = "continuous"
quantile_threshold_list = [0.1, 0.5] # Controls the proportion of the lowest popularity vehicle types to be dropped
imputation_multiplier_list = [2, 3, 4] # Controls the number of times the dataframe records are duplicated

In [6]:
for target_type in target_type_list:
    for quantile_threshold in quantile_threshold_list:
        for imputation_multiplier in imputation_multiplier_list:
            
            vehicle_presence_df = DataframeBuilder.vehicle_presence(presence_type=presence_type, vehicle_type="year_model", target_type=target_type)
            
            # Create additional records based on the imputation multiplier
            vehicle_presence_df = pd.concat([vehicle_presence_df] * imputation_multiplier, ignore_index=True)
            
            if presence_type == "continuous":
                
              
                if 'TotalPrice' in vehicle_presence_df.columns:
                    sums = vehicle_presence_df.drop(columns=['TotalPrice', 'Date']).sum()
                    
                elif 'TotalPartsSold' in vehicle_presence_df.columns:
                    sums = vehicle_presence_df.drop(columns=['TotalPartsSold', 'Date']).sum()
                
                
                try:
                    # Determine the threshold for the bottom x% of sums
                    threshold = sums.quantile(quantile_threshold)
                
            
                    # Find columns to drop
                    cols_to_drop = sums[sums <= threshold].index.tolist()
            
                    # Drop columns from the dataframe
                    vehicle_presence_df = vehicle_presence_df.drop(columns=cols_to_drop)
                    
                except ValueError:
                    print("Sums not found, TotalPrice and TotalPartsSold are not available in vehicle_presence_df.")
                    
            vehicle_presence_df.columns = vehicle_presence_df.columns.astype(str)
            
            # Create the machine learning model steps here, including training and testing LR
            # Separating features and target
            X = vehicle_presence_df.drop(columns=[target_type, 'Date'])
            y = vehicle_presence_df[target_type]
            
            # Normalize the features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            # Split the data for training and testing
            X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
            
            # Initialize and fit the Lasso regression model
            lasso = Lasso()
            
            
            # Define the parameter grid
            param_grid = {'alpha': [1e-2, 1e-1, 1, 10, 100]}
            
            # Use GridSearchCV for tuning
            grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_percentage_error', return_train_score=True)
            grid_search.fit(X_train, y_train)
            
            # Get the results
            results = grid_search.cv_results_
            
            # Create a DataFrame to display results
            results_df = pd.DataFrame({
                'alpha': results['param_alpha'],
                'mean_test_score': results['mean_test_score'],
                'std_test_score': results['std_test_score']
            })
            
            results_df.to_csv(f'{target_type}_{quantile_threshold}_{imputation_multiplier}_LASSO_Results.csv', index=False)

C:\Users\aljo9\PycharmProjects\NV_Picapart_Analysis\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.220e+06, tolerance: 1.005e+05
  model = cd_fast.enet_coordinate_descent(
C:\Users\aljo9\PycharmProjects\NV_Picapart_Analysis\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.244e+06, tolerance: 9.480e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\aljo9\PycharmProjects\NV_Picapart_Analysis\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, chec